In [1]:
#import packages
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [2]:
#read in list of EU countries
eu_df = pd.read_csv('./additional_data/EU27_COUNTRY_LIST.csv')

In [3]:
#function for cleaning sheets in the violence dataset - dropping unncessary columns and rows, and 
#then filtering this by merging with the EU country list DF.
def clean_violence_sheet(file, sheet_name):
    #read in dataset
    df = pd.read_excel(file, sheet_name)
    #drop unnecessary rows
    df.drop(df.index[0:9], inplace=True)
    #rename columns
    df = df.rename(columns=df.iloc[0])
    #drop further rows
    df.drop(df.index[0:3], inplace=True)
    #change name of first column to Country to fit into tidy data format
    df = df.rename(columns={"TIME": "Country"})
    #reformat names of countries - Czech Republic and Germany - to fit into standardized formatting across sheets
    df = df.replace({'Country' : { "Czechia" : "Czech Republic", "Germany (until 1990 former territory of the FRG)" : "Germany"}})
    #merge new df with the eu countries df - filters for only EU 27
    merged_df = eu_df.merge(df, on='Country', how='left')
    #drop the 'Initial' column contained in the EU sheet - not needed here
    merged_df = merged_df.drop('Initial', axis = 1)
    #melt df into tidy data format by stacking and restacking - create a 'Value' column to store the respective value in
    merged_df = merged_df.set_index(['Country']).rename_axis(['Year'],axis=1).stack().reset_index()
    merged_df = merged_df.rename(columns={merged_df.columns[2]: "Value"})
    return merged_df

In [4]:
#cleaning three datasets - homicide, rape, and sexual assault 
#only one column / data point for each - number of female victims per hundred thousand inhabitants
#therefore comparable across countries. No context given with NaN values. 
pht_hom_df = clean_violence_sheet('./datasets_raw/violence_master.xlsx', 'Sheet 18')
pht_ra_df = clean_violence_sheet('./datasets_raw/violence_master.xlsx', 'Sheet 36')
pht_sa_df = clean_violence_sheet('./datasets_raw/violence_master.xlsx', 'Sheet 54')

In [5]:
#ensure formatting is in tidy data form - check heads and random samples to ensure consistent
#formatting
pht_hom_df.head()
pht_ra_df.head()
pht_sa_df.head()
pht_hom_df.sample(5)
pht_ra_df.sample(5)
pht_sa_df.sample(5)

,Country,Year,Value
10,Belgium,2018,:
7,Belgium,2015,:
275,Portugal,2010,21.18
108,Spain,2012,24.72
21,Bulgaria,2016,10.53


In [6]:
#count number of null values in each df. Since null values are stored as ':' sum across the values column
pht_hom_df.info()
(pht_hom_df['Value']==':').sum()

pht_sa_df.info()
(pht_sa_df['Value']==':').sum()

pht_ra_df.info()
(pht_ra_df['Value']==':').sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  351 non-null    object
 1   Year     351 non-null    object
 2   Value    351 non-null    object
dtypes: object(3)
memory usage: 8.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  351 non-null    object
 1   Year     351 non-null    object
 2   Value    351 non-null    object
dtypes: object(3)
memory usage: 8.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  351 non-null    object
 1   Year     351 non-null    object
 2   Value    351 non-null    object
dtypes: object(3)
memory usage: 8.4+ KB


112

In [7]:
#there are a reasonable number of null values in many. since this is time series data, 
#we are ok to include these data points, but we will change their value to 0, as this can then
#be filtered out in our visual representation. Average values will not include 0 values, and neither will
#index values
pht_hom_df = pht_hom_df.replace(':', 0)
pht_ra_df = pht_ra_df.replace(':', 0)
pht_sa_df = pht_sa_df.replace(':', 0)
pht_hom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  351 non-null    object 
 1   Year     351 non-null    object 
 2   Value    351 non-null    float64
dtypes: float64(1), object(2)
memory usage: 8.4+ KB


In [8]:
#Compare if each country has the same number of rows
countries = list(eu_df['Country'])
for c in countries:
        print('Country: ' + str(c) + ': '+ str((pht_hom_df['Country'] == c).sum()))
        print('Country: ' + str(c) + ': '+ str((pht_ra_df['Country'] == c).sum()))
        print('Country: ' + str(c) + ': '+ str((pht_sa_df['Country'] == c).sum()))

Country: European Union: 0
Country: European Union: 0
Country: European Union: 0
Country: Belgium: 13
Country: Belgium: 13
Country: Belgium: 13
Country: Bulgaria: 13
Country: Bulgaria: 13
Country: Bulgaria: 13
Country: Czech Republic: 13
Country: Czech Republic: 13
Country: Czech Republic: 13
Country: Denmark: 13
Country: Denmark: 13
Country: Denmark: 13
Country: Germany: 13
Country: Germany: 13
Country: Germany: 13
Country: Estonia: 13
Country: Estonia: 13
Country: Estonia: 13
Country: Ireland: 13
Country: Ireland: 13
Country: Ireland: 13
Country: Greece: 13
Country: Greece: 13
Country: Greece: 13
Country: Spain: 13
Country: Spain: 13
Country: Spain: 13
Country: France: 13
Country: France: 13
Country: France: 13
Country: Croatia: 13
Country: Croatia: 13
Country: Croatia: 13
Country: Italy: 13
Country: Italy: 13
Country: Italy: 13
Country: Cyprus: 13
Country: Cyprus: 13
Country: Cyprus: 13
Country: Latvia: 13
Country: Latvia: 13
Country: Latvia: 13
Country: Lithuania: 13
Country: Lithu

In [9]:
#convert dataframe year dates to datetime - correct format - and check that each year has 
#the same number of rows - 27
pht_ra_df['Year'] = pd.to_datetime(pht_ra_df['Year']).dt.year
pht_sa_df['Year'] = pd.to_datetime(pht_sa_df['Year']).dt.year
pht_hom_df['Year'] = pd.to_datetime(pht_hom_df['Year']).dt.year

for i in range(2000,2025):
    print('Year: ' + str(i) + ': '+ str((pht_ra_df['Year'] == i).sum()))
    print('Year: ' + str(i) + ': '+ str((pht_sa_df['Year'] == i).sum()))
    print('Year: ' + str(i) + ': '+ str((pht_hom_df['Year'] == i).sum()))
    

Year: 2000: 0
Year: 2000: 0
Year: 2000: 0
Year: 2001: 0
Year: 2001: 0
Year: 2001: 0
Year: 2002: 0
Year: 2002: 0
Year: 2002: 0
Year: 2003: 0
Year: 2003: 0
Year: 2003: 0
Year: 2004: 0
Year: 2004: 0
Year: 2004: 0
Year: 2005: 0
Year: 2005: 0
Year: 2005: 0
Year: 2006: 0
Year: 2006: 0
Year: 2006: 0
Year: 2007: 0
Year: 2007: 0
Year: 2007: 0
Year: 2008: 27
Year: 2008: 27
Year: 2008: 27
Year: 2009: 27
Year: 2009: 27
Year: 2009: 27
Year: 2010: 27
Year: 2010: 27
Year: 2010: 27
Year: 2011: 27
Year: 2011: 27
Year: 2011: 27
Year: 2012: 27
Year: 2012: 27
Year: 2012: 27
Year: 2013: 27
Year: 2013: 27
Year: 2013: 27
Year: 2014: 27
Year: 2014: 27
Year: 2014: 27
Year: 2015: 27
Year: 2015: 27
Year: 2015: 27
Year: 2016: 27
Year: 2016: 27
Year: 2016: 27
Year: 2017: 27
Year: 2017: 27
Year: 2017: 27
Year: 2018: 27
Year: 2018: 27
Year: 2018: 27
Year: 2019: 27
Year: 2019: 27
Year: 2019: 27
Year: 2020: 27
Year: 2020: 27
Year: 2020: 27
Year: 2021: 0
Year: 2021: 0
Year: 2021: 0
Year: 2022: 0
Year: 2022: 0
Year: 202

In [10]:
#check summary stats of each df 
print(pht_ra_df.describe())
print(pht_sa_df.describe())
print(pht_hom_df.describe())

              Year       Value
count   351.000000  351.000000
mean   2014.000000   13.853447
std       3.746999   25.884782
min    2008.000000    0.000000
25%    2011.000000    0.000000
50%    2014.000000    4.630000
75%    2017.000000   14.120000
max    2020.000000  161.090000
              Year       Value
count   351.000000  351.000000
mean   2014.000000   20.666638
std       3.746999   33.498719
min    2008.000000    0.000000
25%    2011.000000    0.000000
50%    2014.000000    6.780000
75%    2017.000000   28.520000
max    2020.000000  203.980000
              Year       Value
count   351.000000  351.000000
mean   2014.000000    0.758034
std       3.746999    0.820271
min    2008.000000    0.000000
25%    2011.000000    0.310000
50%    2014.000000    0.600000
75%    2017.000000    0.910000
max    2020.000000    4.700000


In [11]:
#from https://ec.europa.eu/eurostat/cache/metadata/en/crim_esms.htm#coher_compar1654674641738 -
#Comparability across geographies is affected by many factors, such as differences in legal systems, legal definitions, 
#organisation of police,  statistical populations and so on. Therefore not easy to compare - which is why we choose
#to take an average of three statistics in an effort to reduce variability. Comparability over time may also vary due to
#same reasons

#we are going to take an average of the three datasets as this is more representative, so create a master df - copy of one of the existing dfs
master_violence_pht_df = pht_sa_df.copy()
#dropping value - we will replace this with existing values from other dfs
master_violence_pht_df= master_violence_pht_df.drop('Value', axis = 1)
master_violence_pht_df['Homicide_female_victims'] = pht_hom_df['Value']
master_violence_pht_df['Rape_female_victims'] = pht_ra_df['Value']
master_violence_pht_df['SexualAssault_female_victims'] = pht_sa_df['Value']
#create a count variable to count the number of populated columns (i.e. data points that are not null and therefore equal 0) -
#this will be used to calculate respective averages - i.e. we only divide by the total of actual values
master_violence_pht_df['NonZeroCount'] = master_violence_pht_df.iloc[:,2:5].gt(0).sum(axis=1)
#calculate an average of the three values
master_violence_pht_df['Average_no_female_victims_per_100k'] = (pht_sa_df['Value'] + pht_ra_df['Value'] + pht_hom_df['Value']) / master_violence_pht_df['NonZeroCount']
master_violence_pht_df = master_violence_pht_df.drop('NonZeroCount', axis = 1)
#replace NaN values with 0
master_violence_pht_df['Average_no_female_victims_per_100k'] = master_violence_pht_df['Average_no_female_victims_per_100k'].replace(np.nan, 0)

In [12]:
#index calculation - our assumptions are the minimum and maximum values that make up the scale for calculating an index. 
#inspired by https://ourworldindata.org/human-development-index#:~:text=The%20HDI%20is%20calculated%20as,and%20expected%20years%20of%20schooling). 
# here we have chosen the worst value as a the 'maximum' worst average of the three forms of violence
#this is because finding specific data around 'natural zeros' - as explained here - https://hdr.undp.org/sites/default/files/data/2020/hdr2016_technical_notes.pdf -
#is difficult to find with violence data 
worst_value = (master_violence_pht_df['Homicide_female_victims'].max() + master_violence_pht_df['Rape_female_victims'].max() + master_violence_pht_df['SexualAssault_female_victims'].max()) / 3
best_value = 0

for index, row in master_violence_pht_df.iterrows():
    if row['Average_no_female_victims_per_100k'] != 0:
        master_violence_pht_df.loc[index, 'IndexValueViolence'] = (row['Average_no_female_victims_per_100k'] - worst_value) / (best_value - worst_value)
    else:
        master_violence_pht_df.loc[index, 'IndexValueViolence'] = 0

In [13]:
master_violence_pht_df

,Country,Year,Homicide_female_victims,Rape_female_victims,SexualAssault_female_victims,Average_no_female_victims_per_100k,IndexValueViolence
0,Belgium,2008,0.00,0.00,0.00,0.000000,0.000000
1,Belgium,2009,0.00,0.00,0.00,0.000000,0.000000
2,Belgium,2010,0.00,0.00,0.00,0.000000,0.000000
3,Belgium,2011,0.00,0.00,0.00,0.000000,0.000000
4,Belgium,2012,0.00,0.00,0.00,0.000000,0.000000
...,...,...,...,...,...,...,...
346,Sweden,2016,0.59,118.27,197.27,105.376667,0.145063
347,Sweden,2017,0.54,129.23,203.98,111.250000,0.097412
348,Sweden,2018,0.66,140.46,197.99,113.036667,0.082916
349,Sweden,2019,0.49,150.13,192.93,114.516667,0.070909


In [14]:
master_violence_pht_df.to_csv('./datasets_cleaned/masterviolence_pht_df.csv')